<a href="https://colab.research.google.com/github/AafreenFathimaJ/AICTE-INTERNSHIP/blob/main/Week_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from google.colab import files
import zipfile
import os

In [3]:
uploaded = files.upload()


Saving garbage.zip to garbage.zip


In [6]:
zip_path = list(uploaded.keys())[0]  # Get the uploaded filename
dataset_path = "/content/garbage/garbage"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

print("✅ Dataset extracted!")

✅ Dataset extracted!


In [7]:
image_size = (124, 124)
batch_size = 32
data_dir = dataset_path + "/TrashType_Image_Dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("✅ Classes:", class_names)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
✅ Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [8]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [9]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [10]:
base_model = tf.keras.applications.EfficientNetV2B0(include_top=False, input_shape=(124, 124, 3), weights="imagenet")
base_model.trainable = False

model = tf.keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 4, 4, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,919,312 (22.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 5,919,312 (22.58 MB)

In [11]:
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 71s 841ms/step - accuracy: 0.2025 - loss: 1.8200 - val_accuracy: 0.2455 - val_loss: 1.7255
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 75s 734ms/step - accuracy: 0.2166 - loss: 1.7308 - val_accuracy: 0.2040 - val_loss: 1.7277
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 82s 729ms/step - accuracy: 0.2245 - loss: 1.7299 - val_accuracy: 0.2455 - val_loss: 1.7250
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 82s 735ms/step - accuracy: 0.2183 - loss: 1.7354 - val_accuracy: 0.2455 - val_loss: 1.7301
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 79s 687ms/step - accuracy: 0.2234 - loss: 1.7356 - val_accuracy: 0.2455 - val_loss: 1.7266
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 84s 719ms/step - accuracy: 0.2116 - loss: 1.7371 - val_accuracy: 0.2455 - val_loss: 1.7298
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 82s 733ms/step - accuracy: 0.2242 - loss: 1.7324 - val_accuracy: 0.2455 - val_loss: 1.7268
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 82s 733ms/step - accuracy: 0.2282 - loss: 1.7349 - val_accu

In [12]:
model.save("garbage_model.h5")
print("✅ Model saved as garbage_model.h5")

✅ Model saved as garbage_model.h5


In [13]:
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

def predict_from_image(img_path):
    img = image.load_img(img_path, target_size=(124, 124))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    predicted_class = class_names[class_index]
    print(f"🔍 Predicted: {predicted_class}")
    return predicted_class


In [15]:
uploaded = files.upload()  # Upload an image like glass.jpg
image_filename = list(uploaded.keys())[0]
predict_from_image(image_filename)


Saving paper_400.jpg to paper_400.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
🔍 Predicted: paper


'paper'